In [8]:
import json


def init():
    f = open('./CAIL_data/law.txt', 'r', encoding = 'utf8')
    law = {}
    lawname = {}
    line = f.readline()
    while line:
        lawname[len(law)] = line.strip()
        law[line.strip()] = len(law)
        line = f.readline()
    f.close()


    f = open('./CAIL_data/accu.txt', 'r', encoding = 'utf8')
    accu = {}
    accuname = {}
    line = f.readline()
    while line:
        accuname[len(accu)] = line.strip()
        accu[line.strip()] = len(accu)
        line = f.readline()
    f.close()


    return law, accu, lawname, accuname


law, accu, lawname, accuname = init()

In [ ]:
def getClassNum(kind):
    global law
    global accu

    if kind == 'law':
        return len(law)
    if kind == 'accu':
        return len(accu)

In [7]:
def getName(index, kind):
    global lawname
    global accuname
    if kind == 'law':
        return lawname[index]

    if kind == 'accu':
        return accuname[index]


def gettime(time):
    #将刑期用分类模型来做
    v = int(time['imprisonment'])

    if time['death_penalty']:
        return 0
    if time['life_imprisonment']:
        return 1
    elif v > 10 * 12:
        return 2
    elif v > 7 * 12:
        return 3
    elif v > 5 * 12:
        return 4
    elif v > 3 * 12:
        return 5
    elif v > 2 * 12:
        return 6
    elif v > 1 * 12:
        return 7
    else:
        return 8


def getlabel(d, kind):
    global law
    global accu    
    # 做单标签
    if kind == 'law':
    # 返回多个类的第一个
        return law[str(d['meta']['relevant_articles'][0])]
    if kind == 'accu':
        return accu[d['meta']['accusation'][0]]    
    if kind == 'time':
        return gettime(d['meta']['term_of_imprisonment'])
    
    return label
    


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF
import json
#from predictor import data
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
import pickle
#import thulac


dim = 300

# 斯坦福分词，选取有意义的名词，专有名词，和动词
from stanfordcorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('/home/jeshe/stanford-corenlp-full-2017-06-09/', lang='zh')

def seg_document(document):               
    result = [i for (i, j) in nlp.pos_tag(document) if j in ['NN', 'NR', 'VV']]
    return result

def cut_text(alltext):
#     count = 0
#     #cut = thulac.thulac(seg_only = True)
#     train_text = []
#     for text in alltext:
#         count += 1
#         if count % 2000 == 0:
#             print(count)
#         #train_text.append(cut.cut(text, text = True))
#         train_text.append(seg_document(text))
#     nlp.close()
    with open('./corpus2.txt','r') as f:
        data = f.read()
        f.close()
        train_text = data.split('\n')
     
    return train_text


def train_tfidf(train_data):
    tfidf = TFIDF(
            min_df = 5,
            max_features = dim,
            ngram_range = (1, 3),
            use_idf = 1,
            smooth_idf = 1
            )
    tfidf.fit(train_data)
    
    return tfidf



def read_trainData(path):
    fin = open(path, 'r', encoding = 'utf8')
    
    alltext = []
    
    accu_label = []
    law_label = []
    time_label = []

    line = fin.readline()
    while line:
        d = json.loads(line)
        alltext.append(d['fact'])
        accu_label.append(getlabel(d, 'accu'))
        law_label.append(getlabel(d, 'law'))
        time_label.append(getlabel(d, 'time'))
        line = fin.readline()
    fin.close()

    return alltext, accu_label, law_label, time_label


def train_SVC(vec, label):
    SVC = LinearSVC()
    SVC.fit(vec, label)
    return SVC


if __name__ == '__main__':
    print('reading...')
    alltext, accu_label, law_label, time_label = read_trainData('./CAIL_data/data_train.json')
    print('cut text...')
    train_data = cut_text(alltext)
    print('train tfidf...')
    tfidf = train_tfidf(train_data)
    
    vec = tfidf.transform(train_data)
    
    print('accu SVC')
    accu = train_SVC(vec, accu_label)
    print('law SVC')
    law = train_SVC(vec, law_label)
    print('time SVC')
    time = train_SVC(vec, time_label)

    print('saving model')
    joblib.dump(tfidf, 'predictor/model/tfidf.model')
    joblib.dump(accu, 'predictor/model/accu.model')
    joblib.dump(law, 'predictor/model/law.model')
    joblib.dump(time, 'predictor/model/time.model')

In [ ]:
############################################################################################################################

In [ ]:
from sklearn.externals import joblib
accumodel = joblib.load('predictor/model/accu.model')
lawmodel = joblib.load('predictor/model/law.model')
timemodel = joblib.load('predictor/model/time.model')
tfidf = joblib.load('predictor/model/tfidf.model')

In [ ]:
vec = tfidf.transform(['公诉 机关 起诉 指控 ， 被告人 张 某某 秘密 窃取 他 人财物 ， 价值 2210 元 ， 盗窃 数额较大 ， 其 行为 已触犯 《 中华人民共和国 刑法 》 × × 之 规定 ， 应当 以 × × 追究其 刑事责任 。 建议 判处 被告人 张 某某 × × 以下 刑罚 ， 并 处罚金 '])

In [ ]:
vec.shape

In [ ]:
accumodel.predict(vec)

In [ ]:
list(accu.keys())[list(accu.values()).index(189)]

In [ ]:
##################################################################################################################

In [1]:
import json
#import thulac
from sklearn.externals import joblib

# 斯坦福分词，选取有意义的名词，专有名词，和动词
# from stanfordcorenlp import StanfordCoreNLP
# nlp = StanfordCoreNLP('/home/jeshe/stanford-corenlp-full-2017-06-09/', lang='zh')

# def seg_document(document):               
#     result = [i for (i, j) in nlp.pos_tag(document) if j in ['NN', 'NR', 'VV']]
#     return result

import jieba
class Predictor(object):
    def __init__(self):
        self.tfidf = joblib.load('predictor/model/tfidf.model')
        self.law = joblib.load('predictor/model/law.model')
        self.accu = joblib.load('predictor/model/accu.model')
        self.time = joblib.load('predictor/model/time.model')
        self.batch_size = 1
        
#         self.accumodel = joblib.load('predictor/model/accu.model')
        
        #self.cut = thulac.thulac(seg_only = True)
        


    def predict_law(self, vec):
        y = self.law.predict(vec)
        return [y[0] + 1]
    
    def predict_accu(self, vec):
        y = self.accu.predict(vec)
        return [y[0] + 1]
    
    def predict_time(self, vec):

        y = self.time.predict(vec)[0]
        
        #返回每一个罪名区间的中位数
        if y == 0:
            return -2
        if y == 1:
            return -1
        if y == 2:
            return 120
        if y == 3:
            return 102
        if y == 4:
            return 72
        if y == 5:
            return 48
        if y == 6:
            return 30
        if y == 7:
            return 18
        else:
            return 6
    
    
    def predict(self, content):
        fact = []
        #print('text cut...')
        #print(content)
        for text in content:
            fact.append(' '.join(jieba.cut(text)))
        #print(fact)
#         fact = self.cut.cut(content[0], text = True)
#         print(fact[0])
        vec = self.tfidf.transform(fact)
        ans = {}

        ans['accusation'] = self.predict_accu(vec)
        ans['articles'] = self.predict_law(vec)
        ans['imprisonment'] = self.predict_time(vec)
        
        #print(ans)
        return [ans]


In [ ]:
# inf = open(os.path.join(data_path, 'data_valid.json'), "r", encoding = 'utf8')
# ouf = open(os.path.join(output_path, 'out.txt'), "w", encoding = 'utf8')
# fact = []
# for line in inf.readlines(): 
#     d = json.loads(line)
#     fact.append(d['fact'])   
#     if len(fact)==1:
#         result = solve(fact)
#         cnt += len(result)
#         fact = []
# inf.close()                  

In [3]:
import json
import os
import multiprocessing

#from predictor import Predictor

data_path = "./input/"  # The directory of the input data
output_path = "./output/"  # The directory of the output data


def format_result(result):
    rex = {"accusation": [], "articles": [], "imprisonment": -3}

    res_acc = []
    for x in result["accusation"]:
        if not (x is None):
            res_acc.append(int(x))
    rex["accusation"] = res_acc

    if not (result["imprisonment"] is None):
        rex["imprisonment"] = int(result["imprisonment"])
    else:
        rex["imprisonment"] = -3

    res_art = []
    for x in result["articles"]:
        if not (x is None):
            res_art.append(int(x))
    rex["articles"] = res_art

    return rex


if __name__ == "__main__":
    user = Predictor()
    cnt = 0


    def get_batch():
        v = user.batch_size
        if not (type(v) is int) or v <= 0:
            raise NotImplementedError

        return v


    def solve(fact):
        result = user.predict(fact)

        for a in range(0, len(result)):
            result[a] = format_result(result[a])

        return result


    for file_name in os.listdir(data_path):
        inf = open(os.path.join(data_path, file_name), "r")
        ouf = open(os.path.join(output_path, file_name), "w")

        fact = []

        for line in inf:
            fact.append(json.loads(line)["fact"])
            if len(fact) == get_batch():
                result = solve(fact)
                cnt += len(result)
                for x in result:
                    print(json.dumps(x), file=ouf)
                fact = []

        if len(fact) != 0:
            result = solve(fact)
            cnt += len(result)
            for x in result:
                print(json.dumps(x), file=ouf)
            fact = []

        ouf.close()


Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.035 seconds.
Prefix dict has been built succesfully.


In [ ]:
##################################

In [ ]:


def getlabel(d, kind):
    global law
    global accu    
    # 做单标签
    if kind == 'law':
    # 返回多个类的第一个
        return law[str(d['meta']['relevant_articles'][0])]
    if kind == 'accu':
        return accu[d['meta']['accusation'][0]]    
    if kind == 'time':
        return gettime(d['meta']['term_of_imprisonment'])
    
    return label
    

In [ ]:
def read_validData(path):
    fin = open(path, 'r', encoding = 'utf8')
    
    alltext = []
    
    accu_label_truth = []
    law_label_truth = []
    time_label_truth = []

    line = fin.readline()
    while line:
        d = json.loads(line)
        alltext.append(d['fact'])
        accu_label_truth.append(getlabel(d, 'accu'))
        law_label_truth.append(getlabel(d, 'law'))
        time_label_truth.append(getlabel(d, 'time'))
        line = fin.readline()
    fin.close()

    return alltext, accu_label_truth, law_label_truth, time_label_truth

In [ ]:
alltext, accu_label_truth, law_label_truth, time_label_truth = read_validData('./CAIL_data/data_valid.json')

In [ ]:
jud = Judger('./CAIL_data/accu.txt', './CAIL_data/law.txt')

In [ ]:
valid_x0 = [] # 案情描述，事实部分
valid_y01 = [] # imprisonment 刑期类型及长短
valid_y02 = [] # 被告人，背叛罪名，金钱惩罚，相关法律条文

for line in open('./CAIL_data/data_valid.json', 'r'):
    item = json.loads(line)
    #train_x0.append(item['fact'])    # 案情描述，事实部分 
    imprisonment = item['meta'].pop('term_of_imprisonment') # imprisonment 刑期类型及长短
    valid_y01.append(imprisonment)
    valid_y02.append(item['meta']) # 被告人，背叛罪名，金钱惩罚，相关法律条文

In [ ]:
valid_df = pd.concat([pd.DataFrame(valid_y01), pd.DataFrame(valid_y02)], axis=1)
truth = valid_df.copy()

In [ ]:
s2 = set()
for name in truth["accusation"]:
    s2.add(self.accu_dic[name.replace("[", "").replace("]", "")])

In [ ]:
from math import log
import os
import json


class Judger:
    # Initialize Judger, with the path of accusation list and law articles list
    def __init__(self, accusation_path, law_path):
        self.accu_dic = {}

        f = open(accusation_path, "r")
        self.task1_cnt = 0
        for line in f:
            self.task1_cnt += 1
            self.accu_dic[line[:-1]] = self.task1_cnt

        self.law_dic = {}
        f = open(law_path, "r")
        self.task2_cnt = 0
        for line in f:
            self.task2_cnt += 1
            self.law_dic[int(line[:-1])] = self.task2_cnt

    # Format the result generated by the Predictor class
    @staticmethod
    def format_result(result):
        rex = {"accusation": [], "articles": [], "imprisonment": -3}

        res_acc = []
        for x in result["accusation"]:
            if not (x is None):
                res_acc.append(int(x))
        rex["accusation"] = res_acc

        if not (result["imprisonment"] is None):
            rex["imprisonment"] = int(result["imprisonment"])
        else:
            rex["imprisonment"] = -3

        res_art = []
        for x in result["articles"]:
            if not (x is None):
                res_art.append(int(x))
        rex["articles"] = res_art

        return rex

    # Gen new results according to the truth and users output
    def gen_new_result(self, result, truth, label):
        s1 = set(label["accusation"])
        s2 = set()
        for name in truth["accusation"]:
            s2.add(self.accu_dic[name.replace("[", "").replace("]", "")])

        for a in range(0, self.task1_cnt):
            in1 = (a + 1) in s1
            in2 = (a + 1) in s2
            if in1:
                if in2:
                    result[0][a]["TP"] += 1
                else:
                    result[0][a]["FP"] += 1
            else:
                if in2:
                    result[0][a]["FN"] += 1
                else:
                    result[0][a]["TN"] += 1

        s1 = set(label["articles"])
        s2 = set()
        for name in truth["relevant_articles"]:
            s2.add(self.law_dic[name])

        for a in range(0, self.task2_cnt):
            in1 = (a + 1) in s1
            in2 = (a + 1) in s2
            if in1:
                if in2:
                    result[1][a]["TP"] += 1
                else:
                    result[1][a]["FP"] += 1
            else:
                if in2:
                    result[1][a]["FN"] += 1
                else:
                    result[1][a]["TN"] += 1

        result[2]["cnt"] += 1
        sc = 0
        if truth["term_of_imprisonment"]["death_penalty"]:
            if label["imprisonment"] == -2:
                sc = 1
        elif truth["term_of_imprisonment"]["life_imprisonment"]:
            if label["imprisonment"] == -1:
                sc = 1
        else:
            if label["imprisonment"] < 0:
                sc = 0
            else:
                v1 = truth["term_of_imprisonment"]["imprisonment"]
                v2 = label["imprisonment"]
                v = abs(log(v1 + 1) - log(v2 + 1))
                if v <= 0.2:
                    sc = 1
                elif v <= 0.4:
                    sc = 0.8
                elif v <= 0.6:
                    sc = 0.6
                elif v <= 0.8:
                    sc = 0.4
                elif v <= 1.0:
                    sc = 0.2
                else:
                    sc = 0
        sc = sc * 1.0
        result[2]["score"] += sc

        return result

    # Calculate precision, recall and f1 value
    # According to https://github.com/dice-group/gerbil/wiki/Precision,-Recall-and-F1-measure
    @staticmethod
    def get_value(res):
        if res["TP"] == 0:
            if res["FP"] == 0 and res["FN"] == 0:
                precision = 1.0
                recall = 1.0
                f1 = 1.0
            else:
                precision = 0.0
                recall = 0.0
                f1 = 0.0
        else:
            precision = 1.0 * res["TP"] / (res["TP"] + res["FP"])
            recall = 1.0 * res["TP"] / (res["TP"] + res["FN"])
            f1 = 2 * precision * recall / (precision + recall)

        return precision, recall, f1

    # Generate score for the first two subtasks
    def gen_score(self, arr):
        sumf = 0
        y = {"TP": 0, "FP": 0, "FN": 0, "TN": 0}
        for x in arr:
            p, r, f = self.get_value(x)
            sumf += f
            for z in x.keys():
                y[z] += x[z]

        _, __, f_ = self.get_value(y)

        return (f_ + sumf * 1.0 / len(arr)) / 2.0

    # Generatue all scores
    def get_score(self, result):
        s1 = self.gen_score(result[0])
        s2 = self.gen_score(result[1])
        s3 = 1.0 * result[2]["score"] / result[2]["cnt"]
        return [s1, s2, s3]

    # Test with ground truth path and the user's output path
    def test(self, truth_path, output_path):
        cnt = 0
        result = [[], [], {}]
        for a in range(0, self.task1_cnt):
            result[0].append({"TP": 0, "FP": 0, "TN": 0, "FN": 0})
        for a in range(0, self.task2_cnt):
            result[1].append({"TP": 0, "FP": 0, "TN": 0, "FN": 0})
        result[2] = {"cnt": 0, "score": 0}

        for file_name in os.listdir(truth_path):
            inf = open(os.path.join(truth_path, file_name), "r")
            ouf = open(os.path.join(output_path, file_name), "r")

            for line in inf:
                ground_truth = json.loads(line)["meta"]
                user_output = json.loads(ouf.readline())

                cnt += 1
                result = self.gen_new_result(result, ground_truth, user_output)

        return result
